In [4]:
import os
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import numpy as np
# import kaggle
import requests
from io import StringIO
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2

In [55]:
df = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv')
df = df.astype({'date':np.datetime64})

In [149]:
df

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million,month,year
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN,2020-02,2020
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN,2020-02,2020
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN,2020-02,2020
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN,2020-02,2020
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN,2020-02,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164933,ZWE,Africa,Zimbabwe,2022-02-23,234589.0,609.0,339.429,5388.0,0.0,1.286,...,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN,2022-02,2022
164934,ZWE,Africa,Zimbabwe,2022-02-24,234967.0,378.0,338.429,5390.0,2.0,1.286,...,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN,2022-02,2022
164935,ZWE,Africa,Zimbabwe,2022-02-25,235467.0,500.0,348.143,5392.0,2.0,1.000,...,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN,2022-02,2022
164936,ZWE,Africa,Zimbabwe,2022-02-26,235803.0,336.0,368.429,5393.0,1.0,1.000,...,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN,2022-02,2022


In [156]:
df['month'] = df['date'].dt.to_period('M').astype(str)
df['year'] = df['date'].dt.year
case_by_month = df.groupby(['year','month','continent','iso_code','location','life_expectancy','gdp_per_capita']).sum().reset_index()[['month','continent','location','gdp_per_capita','life_expectancy','new_cases']]
case_by_month['new_cases'] = case_by_month['new_cases'].mask(case_by_month['new_cases']<0,0)

In [157]:
case_by_month.sort_values(by='gdp_per_capita',ascending = False)
# case_by_month['total_cases'].round(0)
# case_by_month.sort_values(by=['new_cases'],ascending=True)

,month,continent,location,gdp_per_capita,life_expectancy,new_cases
1818,2020-12,Asia,Qatar,116935.60,80.23,5001.0
1450,2020-10,Asia,Qatar,116935.60,80.23,6796.0
1269,2020-09,Asia,Qatar,116935.60,80.23,6982.0
2753,2021-05,Asia,Qatar,116935.60,80.23,11806.0
2945,2021-06,Asia,Qatar,116935.60,80.23,4613.0
...,...,...,...,...,...,...
3822,2021-11,Africa,Central African Republic,661.24,53.28,190.0
3630,2021-10,Africa,Central African Republic,661.24,53.28,147.0
4204,2022-01,Africa,Central African Republic,661.24,53.28,1756.0
2105,2021-02,Africa,Central African Republic,661.24,53.28,23.0


In [158]:
px.scatter(case_by_month, x="gdp_per_capita", y="life_expectancy", animation_frame="month", animation_group="location",
           size="new_cases", color="continent", hover_name="location",
           log_x=True, size_max=55, range_x=[10000,10000000], range_y=[25,90], 
                     template='plotly_dark')

In [144]:
case_by_month.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4580 entries, 0 to 4579
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   month            4580 non-null   object 
 1   continent        4580 non-null   object 
 2   location         4580 non-null   object 
 3   gdp_per_capita   4580 non-null   float64
 4   life_expectancy  4580 non-null   float64
 5   new_cases        4580 non-null   float64
dtypes: float64(3), object(3)
memory usage: 214.8+ KB


In [96]:
case_by_month = case_by_month.astype({"location":str})
case_by_month.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5563 entries, 0 to 5562
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype    
---  ------       --------------  -----    
 0   month        5563 non-null   period[M]
 1   iso_code     5563 non-null   object   
 2   location     5563 non-null   object   
 3   total_cases  5563 non-null   float64  
dtypes: float64(1), object(2), period[M](1)
memory usage: 174.0+ KB


In [57]:
data

,iso_code,continent,location,date,total_cases,new_cases,total_deaths,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,life_expectancy
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,39835428.0,54.422,18.6,2.581,1.337,1803.987,64.83
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,39835428.0,54.422,18.6,2.581,1.337,1803.987,64.83
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,39835428.0,54.422,18.6,2.581,1.337,1803.987,64.83
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,39835428.0,54.422,18.6,2.581,1.337,1803.987,64.83
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,39835428.0,54.422,18.6,2.581,1.337,1803.987,64.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164933,ZWE,Africa,Zimbabwe,2022-02-23,234589.0,609.0,5388.0,15092171.0,42.729,19.6,2.822,1.882,1899.775,61.49
164934,ZWE,Africa,Zimbabwe,2022-02-24,234967.0,378.0,5390.0,15092171.0,42.729,19.6,2.822,1.882,1899.775,61.49
164935,ZWE,Africa,Zimbabwe,2022-02-25,235467.0,500.0,5392.0,15092171.0,42.729,19.6,2.822,1.882,1899.775,61.49
164936,ZWE,Africa,Zimbabwe,2022-02-26,235803.0,336.0,5393.0,15092171.0,42.729,19.6,2.822,1.882,1899.775,61.49
